In [4]:
import pandas as pd
from pathlib import Path
train_file_name = "train.parquet"
test_file_name = "test.parquet"
motor_data_dir = "/data/processed_datasets/processed_datasets/ehr_foundation_data/ohdsi_cumc_deid/ohdsi_cumc_deid_2023q4r3_v3_mapped/models/femr/motor_large_no_visit_id_8192/results/AMI/motor_730/features_with_label"
train_parquet = pd.read_parquet(Path(motor_data_dir)/train_file_name)
test_parquet = pd.read_parquet(Path(motor_data_dir)/test_file_name)
# print(train_parquet)
print(test_parquet)

       subject_id            prediction_time  boolean_value  \
0             971 2009-02-06 14:53:00.250199              0   
1             971 2009-05-02 12:29:00.250349              0   
2             971 2009-06-20 14:02:00.250609              0   
3             971 2009-06-28 14:03:00.250609              0   
4             971 2013-04-14 09:30:00.000000              0   
...           ...                        ...            ...   
49995    15615820 2022-01-31 15:14:06.000000              0   
49996    15615820 2022-02-14 13:00:00.000000              0   
49997    15619505 2022-12-18 10:36:53.000000              0   
49998    15623225 2021-05-25 10:46:30.000000              0   
49999    15623225 2022-06-20 10:29:44.000000              0   

                                                features  
0      [-1.5571088, -0.6172047, 1.2579645, 0.30174413...  
1      [-1.246936, -0.7224043, 1.2480731, 0.20652555,...  
2      [-1.229949, -0.69501644, 1.1290375, 0.41305855...  
3      

In [2]:
from pathlib import Path
import pandas as pd

# ─── CONFIGURE ────────────────────────────────────────────────────────────────
base_dir    = "/data/processed_datasets/processed_datasets/ehr_foundation_data/ohdsi_cumc_deid/ohdsi_cumc_deid_2023q4r3_v3_mapped/llama"                  # ← change this
# task_labels = ["AMI","CLL","HTN", "Ischemic_Stroke", "MASLD", "Osteoporosis", "Pancreatic_Cancer" ,"SLE" ,"T2DM"]     # ← your list of task sub-dirs
task_labels=["Celiac"]
# ───────────────────────────────────────────────────────────────────────────────

for task_label in task_labels:
    p = Path(base_dir) / task_label / "features_with_label"

    train_path   = p / "train.parquet"
    tune_path    = p / "tune.parquet"
    testing_path = p / "testing.parquet"
    new_test     = p / "test.parquet"

    # 1) load
    df_train = pd.read_parquet(train_path)
    df_tune  = pd.read_parquet(tune_path)

    # 2) concat & sort
    df = pd.concat([df_train, df_tune], ignore_index=True)
    df = df.sort_values(by=["subject_id", "prediction_time"])

    # 3) overwrite train.parquet
    df.to_parquet(train_path, index=False)

    # 4) delete tune.parquet
    if tune_path.exists():
        tune_path.unlink()

    # 5) rename testing.parquet → test.parquet
    if testing_path.exists():
        testing_path.rename(new_test)

    print(f"[✓] {task_label}: train updated, tune deleted, testing → test")

[✓] Celiac: train updated, tune deleted, testing → test


In [3]:
# raw_data_dir = "/data/processed_datasets/processed_datasets/ehr_foundation_data/ohdsi_cumc_deid/ohdsi_cumc_deid_2023q4r3_v3_mapped/task_labels/phenotype_sample/AMI"
# train_data_parquet = pd.read_parquet(Path(raw_data_dir)/train_file_name)
# print(train_data_parquet)

# tune_data_parquet = pd.read_parquet(Path(raw_data_dir)/tune_file_name)
# print(tune_data_parquet)
import polars as pl
import torch
import numpy as np
base_dir = "/data/processed_datasets/processed_datasets/ehr_foundation_data/ohdsi_cumc_deid/ohdsi_cumc_deid_2023q4r3_v3_mapped/mamba/AMI/features_with_label"
train_path = Path(base_dir)/"train.parquet"
# tune_path = Path(base_dir)/"tune.parquet"
test_path = Path(base_dir)/"test.parquet"
train_df = pl.read_parquet(train_path)
test_df = pl.read_parquet(test_path)
print(train_df)
print(test_df)


shape: (100_000, 4)
┌────────────┬────────────────────────────┬───────────────┬─────────────────────────────────┐
│ subject_id ┆ prediction_time            ┆ boolean_value ┆ features                        │
│ ---        ┆ ---                        ┆ ---           ┆ ---                             │
│ i64        ┆ datetime[ns]               ┆ i64           ┆ list[f32]                       │
╞════════════╪════════════════════════════╪═══════════════╪═════════════════════════════════╡
│ 610        ┆ 2015-01-04 11:00:00        ┆ 0             ┆ [1.23367, -0.172913, … -0.5061… │
│ 610        ┆ 2015-02-09 09:00:00        ┆ 0             ┆ [1.244484, 0.442903, … -0.2181… │
│ 610        ┆ 2015-02-17 12:36:00.100270 ┆ 0             ┆ [0.370727, 0.445867, … -0.7110… │
│ 610        ┆ 2015-02-23 12:00:00.100310 ┆ 0             ┆ [0.969827, 0.637445, … -0.4157… │
│ 610        ┆ 2015-04-05 09:00:00        ┆ 0             ┆ [0.515411, 0.63285, … -0.19573… │
│ …          ┆ …                        

In [3]:
from pathlib import Path
# raw_data_dir = "/data/processed_datasets/processed_datasets/ehr_foundation_data/ohdsi_cumc_deid/ohdsi_cumc_deid_2023q4r3_v3_mapped/post_transform/data/train"
# # raw_outcome = "/data/processed_datasets/processed_datasets/ehr_foundation_data/ohdsi_cumc_deid/ohdsi_cumc_deid_2023q4r3_v3_mapped/task_labels/patient_outcomes_sample/readmission"
# for file in sorted(Path(raw_data_dir).glob("*.parquet")):
#     print(file,Path(file).stem)


a = 124.1313
print(int(a))

# train_data_parquet = pd.read_parquet(Path(raw_data_dir)/"0.parquet")
# raw_path =  pd.read_parquet(Path(raw_outcome)/"train.parquet")
# print(raw_path)

124
